<a href="https://colab.research.google.com/github/Thiago-sa/Busca_Endereco/blob/main/Busca_endereco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Localizando endereços através da API da VIACEP**


In [28]:
#Importação de pacotes
import pandas as pd
import urllib.request
import requests
import json

In [29]:
#Importação do arquivo de dados
df = pd.read_excel('/content/dados.xlsx', dtype={ 'CEP_CLIENTE': 'str'})

# **Vamos fazer uma análise exploratória básica**

O objetivo dessa análise é ter uma visão do conjunto de dados, compreendendo seu tamanho, variáveis, tipo de dados e período de dados coletados.

In [30]:
#Tamanho do conjunto de dados, 29 linhas e 2 colunas
df.shape

(29, 2)

In [31]:
#Visão geral do conjunto de dados, no momento só possui 2 colunas.
df.head()

,CODIGO_CLIENTE,CEP_CLIENTE
0,5695,78795000
1,5705,74905090
2,5711,78725160
3,5727,75904520
4,5994,43900000


In [32]:
# Vamos avaliar os tipos de variaveis
# Precisamos confirmar que a variacel CEP_CLIENTE esteja como String.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CODIGO_CLIENTE  29 non-null     int64 
 1   CEP_CLIENTE     29 non-null     object
dtypes: int64(1), object(1)
memory usage: 592.0+ bytes


In [ ]:
# Vamos criar um DataFrame separado somente com os CEP´s e depois manter somente os CEP´s únicos.
# Dessa forma, consultaremos a API somente uma vez para cada CEP
df_ceps_clientes = df['CEP_CLIENTE']
df_ceps_clientes = pd.DataFrame(df_ceps_clientes)
df_ceps_clientes.drop_duplicates(inplace = True)

df_ceps_clientes.head()

In [34]:
# Visualizando a quantidade de CEP unico
print('Ceps únicos de clientes:' , df_ceps_clientes.count())

Ceps únicos de clientes: CEP_CLIENTE    28
dtype: int64


In [35]:
# Variavel url para armazenar o caminho da API
url = 'https://viacep.com.br/ws/%s/json/' % '04849000'

# Variavel header para armazenar o parametro do header da API
headers = {'User-Agent': 'Autociencia/1.0'}

# Variavel requisição para fazer a chamada na API
requisicao = urllib.request.Request(url, headers=headers, method='GET')

# Variavel cliente para abrir a requisição
cliente = urllib.request.urlopen(requisicao)

# Variavel conteúdo para armazenar o conteúdo da api
conteudo = cliente.read().decode('utf-8')

# Variavel endereco para armazenar o conteúdo da API ja converdio em um dicionário
endereco = json.loads(conteudo)

In [36]:
# Exibindo o endereço completo sem tratamento
print(endereco)

{'cep': '04849-000', 'logradouro': 'Rua Gilberto Freyre', 'complemento': '', 'bairro': 'Parque Residencial Cocaia', 'localidade': 'São Paulo', 'uf': 'SP', 'ibge': '3550308', 'gia': '1004', 'ddd': '11', 'siafi': '7107'}


In [ ]:
# Exibindo os valores de forma separada e organizada
print('Endereco:' , endereco['logradouro'])
print('Bairro:' , endereco['bairro'])
print('Cidade:' , endereco['localidade'])
print('Estado:' , endereco['uf'])

#No códigos abaixo vamos criar novas colunas com as informações:

- Endereço
- Bairro
- Cidade
- Estado


In [ ]:
for idx, row in df_ceps_clientes.iterrows():

  cep = row.CEP_CLIENTE
  url = 'https://viacep.com.br/ws/%s/json/' % cep
  headers = {'User-Agent': 'Autociencia/1.0'}
  requisicao = urllib.request.Request(url, headers=headers, method='GET')
  cliente = urllib.request.urlopen(requisicao)
  conteudo = cliente.read().decode('utf-8')
  endereco = json.loads(conteudo)
  cliente.close()
  print(cep)

  if 'cep' in endereco:
    df_ceps_clientes.at[idx, 'ENDERECO_CLIENTE'] = endereco['logradouro']
    df_ceps_clientes.at[idx, 'BAIRRO_CLIENTE'] = endereco['bairro']
    df_ceps_clientes.at[idx, 'CIDADE_CLIENTE'] = endereco['localidade']
    df_ceps_clientes.at[idx, 'ESTADO_CLIENTE'] = endereco['uf']

  else:
    df_ceps_clientes.at[idx, 'ENDERECO_CLIENTE'] = 'Não localizado'
    df_ceps_clientes.at[idx, 'BAIRRO_CLIENTE'] = 'Não localizado'
    df_ceps_clientes.at[idx, 'CIDADE_CLIENTE'] = 'Não localizado'
    df_ceps_clientes.at[idx, 'ESTADO_CLIENTE'] = 'Não localizado'

In [ ]:
# O código mostra de maneira organizada as colunas criadas no dataframe e as informações de endereçamento do cliente.
df_ceps_clientes.head(25)

 # **Agora está muito mais intuitivo **😯

 O código abaixo permite o usuário(você) a digitar qualquer CEP e achar o endereço completo.

 Basta clicar no botãozinho de play ▶ no canto esquerdo do código, em seguida, colocar os numeros do CEP no campo solicitado.


In [43]:
# <--- Para conseguir colocar o CEP clica no botão de PLAY a esquerda e insira um cep.
def consultar_endereco_por_cep(cep):
    url = f'https://viacep.com.br/ws/{cep}/json/'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if 'erro' not in data:
            return data
        else:
            print('CEP não encontrado.')
    else:
        print('Erro na consulta. Verifique sua conexão com a internet.')

def main():
    while True:
        cep = input('Digite o CEP (somente números): ')
        if len(cep) == 8 and cep.isdigit():
            endereco = consultar_endereco_por_cep(cep)
            if endereco:
                print('\nInformações do endereço:')
                print(f'CEP: {endereco["cep"]}')
                print(f'Logradouro: {endereco["logradouro"]}')
                print(f'Complemento: {endereco["complemento"]}')
                print(f'Bairro: {endereco["bairro"]}')
                print(f'Cidade: {endereco["localidade"]}')
                print(f'Estado: {endereco["uf"]}')
            break
        else:
            print('CEP inválido. O CEP deve conter 8 números.')


if __name__ == '__main__':
    main()


Digite o CEP (somente números): 04849000

Informações do endereço:
CEP: 04849-000
Logradouro: Rua Gilberto Freyre
Complemento: 
Bairro: Parque Residencial Cocaia
Cidade: São Paulo
Estado: SP
